In [12]:
from dotenv import load_dotenv
import os
import dspy
import pandas as pd
load_dotenv()

True

In [2]:
lm = dspy.LM('openai/gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'))
dspy.configure(lm=lm)


lm("What year is it?", temperature=0.7) 

['The current year is 2023.']

Signatures

Strukturera vad LMM kommer göra:

In [3]:
class Chat(dspy.Signature):
    "You are a helpful assistant that answer questions about cats and dogs."
    
    question: str = dspy.InputField(desc="Questions asked by the user")
    response: str = dspy.OutputField(desc="Response to the question")

Modules:

Här programmerar vi olika prompting tekniker till våra signatures istället för att skriva dom:

In [4]:
class Model(dspy.Module):
    def __init__(self):
        super().__init__()
        self.respond = dspy.Predict(Chat)

    def forward(self, question: str):
        return self.respond(question=question)

In [5]:
model = Model()
response = model(question="Why is the earth round?")

In [6]:
lm.inspect_history()





[2025-05-18T12:03:44.224607]

System message:

Your input fields are:
1. `question` (str): Questions asked by the user

Your output fields are:
1. `response` (str): Response to the question

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## response ## ]]
{response}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        You are a helpful assistant that answer questions about cats and dogs.


User message:

[[ ## question ## ]]
Why is the earth round?

Respond with the corresponding output fields, starting with the field `[[ ## response ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## response ## ]]
The Earth is round due to the force of gravity. When the planet formed, gravity pulled matter together into a shape that minimizes potential energy, which is a sphere. This shape allows for an even distribution of gravitational force, r

In [7]:
class Model(dspy.Module):
    def __init__(self):
        super().__init__()
        self.respond = dspy.ChainOfThought(Chat)

    def forward(self, question: str):
        return self.respond(question=question)

In [8]:
model = Model()
response = model(question="Why is the earth round?")

In [9]:
lm.inspect_history()





[2025-05-18T12:03:48.028603]

System message:

Your input fields are:
1. `question` (str): Questions asked by the user

Your output fields are:
1. `reasoning` (str)
2. `response` (str): Response to the question

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## response ## ]]
{response}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        You are a helpful assistant that answer questions about cats and dogs.


User message:

[[ ## question ## ]]
Why is the earth round?

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## response ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## reasoning ## ]]
The Earth is round due to the force of gravity. When the planet formed, gravity pulled matter together into a shape that minimizes potential energy, 

Evaluators:

För att mäta hur bra din modell presterar till din task behöver du specificera metrics, men först ha data som den kan evalueras mot - DSPy exapmles

In [13]:
!pip install kagglehub
import kagglehub

path = kagglehub.dataset_download("melissamonfared/mental-health-counseling-conversations-k")

path = path + "/melissamonfared/mental-health-counseling-conversations-k"

  Obtaining dependency information for kagglehub from https://files.pythonhosted.org/packages/49/bf/c2a24567bb6bd80c1fe7cb2ed1a332666476f69c313256aff96094bef93e/kagglehub-0.3.12-py3-none-any.whl.metadata
  Using cached kagglehub-0.3.12-py3-none-any.whl.metadata (38 kB)
Using cached kagglehub-0.3.12-py3-none-any.whl (67 kB)


In [15]:
df = pd.read_json(path)
print(df.head())

/var/folders/3z/kfntmrkd4gq2xvd2qj5rm_s40000gn/T/ipykernel_846/3112260385.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(path)


ValueError: Expected object or value

How well does the system response cover all key facts in the gold response? And the other way around, how well is the system response not saying things that aren't in the gold response? That metric is essentially a semantic F1. This metric is actually implemented as a very simple DSPy module using whatever LM we're working with.
https://github.com/stanfordnlp/dspy/blob/main/dspy/evaluate/auto_evaluation.py#L21
